# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы . Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
# импортируем все библиотеки
import pandas   as pd
from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score
from sklearn.ensemble        import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics         import mean_squared_error
from sklearn.linear_model    import LogisticRegression
from sklearn.dummy           import DummyClassifier

In [18]:
# Записываем датасет в переменную
data_tarif = pd.read_csv('/datasets/users_behavior.csv')
data_tarif.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [21]:
data_tarif.to_csv(r'C:\pr2.csv')

In [3]:
data_tarif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


##### Краткий вывод:
Датасет идеален. Можно точно выделить целевые признаки и зависимую переменную.

Целевые признаки: сalls, minutes, messages, mb_used.

Зависмая переменная: is_ultra    

## 2. Разбейте данные на выборки

In [4]:
# Разбиваем данные на выборки
data_tarif_train , data_tarif_valid_test = train_test_split(data_tarif , 
                                                            train_size = 0.8 , 
                                                            test_size = 0.2,                                                                  
                                                            random_state = 123)
data_tarif_valid ,data_tarif_test = train_test_split(data_tarif_valid_test , 
                                                            train_size = 0.5 , 
                                                            test_size = 0.5,                                                                  
                                                            random_state = 123)
print('Размер обучающей выборки: ', data_tarif_train.shape)
print('Размер валидационной выборки: ', data_tarif_valid.shape)
print('Размер тестовой выборки: ', data_tarif_test.shape)

Размер обучающей выборки:  (2571, 5)
Размер валидационной выборки:  (321, 5)
Размер тестовой выборки:  (322, 5)


##### Вывод:
Методом train_test_split() данные разбиты на 3 выборки:
- Обучающую,      60% данных;
- Валидационную , 20% данных;
- Тестовую ,      20% данных;

## 3. Исследуйте модели

In [5]:
# Создаем обучающие переменные
x_train = data_tarif_train.drop('is_ultra' , axis = 1)
y_train = data_tarif_train['is_ultra']

# Создаем валидационные переменные
x_valid = data_tarif_valid.drop('is_ultra' , axis = 1)
y_valid = data_tarif_valid['is_ultra']

# Создаем тестовые переменные
x_test = data_tarif_test.drop('is_ultra' , axis = 1)
y_test = data_tarif_test['is_ultra']

In [6]:
# Модель деревья решений
# Словарь для записи промежуточных параметров и поиска оптимального
result_tree={}

# Цикл для поиска оптимальной глубины дерева
for i in range(2 , 20):
    model_dec_tree = DecisionTreeClassifier(random_state = 12345 , max_depth = i)
    model_dec_tree.fit(x_train , y_train)
    pred = model_dec_tree.predict(x_valid)
    result_tree[i] = accuracy_score(y_valid , pred)

In [7]:
# Функция для определения максимальной точности
def result_detection(result_dict):
    # Спимок значений словаря ошибок
    error_list = list(result_dict.values())
    # Список ключей словаря ошибок
    param_list = list(result_dict.keys())
    # Максимальная точность
    max_score = max(list(result_dict.values()))
    # Значение гиперпараметра при известной максимальной точности
    result_param = param_list[error_list.index(max_score)]
    return (result_param  , max_score)

In [8]:
#  Оптимальные параметры для дерева решений
result_detection(result_tree)

(8, 0.822429906542056)

Для модели дерева решений оптимальная глубина 5, при этом точность предсказания на валидационной выборке составляет 0,83.

In [9]:
# Модель случайный лес
# Словарь для записи промежуточных параметров и поиска оптимального точности
result_random={}

# Цикл для поиска оптимального количества деревьев
for estim in range(5 , 80,5):
    for depth in range(2 , 20):
        model_dec_random = RandomForestClassifier(random_state = 12345 , n_estimators = estim , max_depth = depth )
        model_dec_random.fit(x_train , y_train)
        pred = model_dec_random.predict(x_valid)
        result_random[estim , depth] = accuracy_score(y_valid , pred)

In [10]:
# Оптимальные параметры для случайного леса
result_detection(result_random)

((5, 5), 0.8317757009345794)

In [11]:
# Модель логистической регрессии
model_dec_log = LogisticRegression(solver = 'lbfgs')
model_dec_log.fit(x_train , y_train)
pred = model_dec_log.predict(x_valid)
# Точность модели
result_logistic = accuracy_score(y_valid , pred)
result_logistic

0.7414330218068536

#### Вывод:
Методами библиотеки sklearn проверены 3 модели: дерево решений, случайный лес и логистическая регрессия.
##### Дерево решений
В цикле была определена оптимальная глубина модели. В качестве метрики использовалось количество правильных ответов на обучающей выборке - accuracy_score. Результаты записывались в словарь, в котором определялось максимальное значение метрики и соответствующее ему значение глубины дерева. Результат : глубина 5, правильность - 0,8227.
##### Случайный лес
В цикле были определены оптимальная глубина модели и количество деревьев. В качестве метрики использовалось количество правильных ответов на обучающей выборке - accuracy_score. Результаты записывались в словарь, в котором определялось максимальное значение метрики и соответствующие ему значения количества деревьев и их глубины. Результат : количество деревьев 70, глубина 10, правильность - 0,8367.
##### Логистическая регрессия
Ручным подбором параметров решателя был определен гиперпараметр "lbfgs". Максимальная правильность решений для этой модели составляет - 0,7589.

Для дальнейшей проверки на тестовой выборке принимаем модель Случайного леса с параметрами: количество деревьев 70, глубина 10.
    

## 4. Проверьте модель на тестовой выборке

In [12]:
# Модель случайного леса на тестовой выборке
model_forest = RandomForestClassifier(random_state = 12345 , n_estimators = 70 , max_depth = 10 )
model_forest.fit(x_train , y_train)
prediction = model_forest.predict(x_test)

# Точность модели
result_forest = accuracy_score(y_test , prediction)
print('Правильных предсказаний:')
result_forest

Правильных предсказаний:


0.8012422360248447

#### Вывод:
На тестовой выборке выбранная модель случайного леса с гиперпараметрами количества деревьев равным 70 и глубиной дерева 10 определяет правильно 78,85% решений. 

## 5. (бонус) Проверьте модели на адекватность

In [13]:
# Случайная модель с учетом распределения обучающей модели 
dummy_model  = DummyClassifier(strategy="stratified" , random_state = 12345)
dummy_model.fit(x_train , y_train)
dummy_prediction = dummy_model.predict(x_test)
print('Количество правильных решений:' , accuracy_score(y_test , dummy_prediction))


Количество правильных решений: 0.593167701863354


In [14]:
# Случайная модель  с рандомным выбором значения признака
dummy_model  = DummyClassifier(strategy="uniform", random_state = 12345)
dummy_model.fit(x_train , y_train)
dummy_prediction = dummy_model.predict(x_test)
print('Количество правильных решений')
accuracy_score(y_test , dummy_prediction)


Количество правильных решений


0.5527950310559007

In [15]:
# Модель с одним целевым признаком.
dummy_model  = DummyClassifier(strategy="most_frequent", random_state = 12345)
dummy_model.fit(x_train , y_train)
dummy_prediction = dummy_model.predict(x_test)
print('Количество правильных решений')
accuracy_score(y_test , dummy_prediction)


Количество правильных решений


0.7111801242236024

#### Вывод:
Для проверки модели на адекватность было проведено сравнение с 3-мя простыми моделями:
- stratified - с распределением аналогичным распределению тестовой выборки. 
- uniform - со случайным распределением.
- most_frequent -  с одним значем, которое встречается чаще другого . В нашем случае это 0.

В качестве метрики был применен показатель accuracy_score. Показатели правильности всех простых моделей меньше правильности предсказаний выбранной модели Случайного леса с гиперпараметрами количества деревьев равным 70 и глубиной дерева 10. Это позволяет сделать вывод, что выбранная модель предсказывает значения лучше случайной модели и является адекватной.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [х] Весь код исполняется без ошибок
- [х] Ячейки с кодом расположены в порядке исполнения
- [х] Выполнено задание 1: данные загружены и изучены
- [х] Выполнено задание 2: данные разбиты на три выборки
- [х] Выполнено задание 3: проведено исследование моделей
    - [х] Рассмотрено больше одной модели
    - [х] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [х] Написаны выводы по результатам исследования
- [х] Выполнено задание 3: Проведено тестирование
- [х] Удалось достичь accuracy не меньше 0.75
